In [ ]:
!pip install langchain-community Chroma Chromadb pymuPDF pyPDF2 sentence-transformers langchain pypdf

In [13]:
from langchain.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader
from PyPDF2 import PdfReader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
import chromadb
import os
import fitz
import shutil
import re
from google.colab import files

In [14]:
# Crear un directorio para cargar los archivos PDF
pdf_directory = '/content/pdf_files'
os.makedirs(pdf_directory, exist_ok=True)

# Cargar archivos PDF
uploaded = files.upload()

# Mover los archivos cargados al directorio especificado
for filename in uploaded.keys():
    os.rename(filename, os.path.join(pdf_directory, filename))

Saving preguntas.pdf to preguntas.pdf
Saving respuestas.pdf to respuestas.pdf
Saving teoria.pdf to teoria.pdf


In [15]:
import os
from langchain_community.document_loaders import PyPDFLoader

# Directorio que contiene los archivos PDF
pdf_directory = '/content/pdf_files'

# Lista para almacenar documentos
all_docs = []

# Iterar sobre todos los archivos en el directorio
for filename in os.listdir(pdf_directory):
    if filename.lower().endswith('.pdf'):
        file_path = os.path.join(pdf_directory, filename)
        print(f"Cargando archivo: {file_path}")

        # Cargar el PDF
        loader = PyPDFLoader(file_path)
        docs = loader.load()

        # Agregar los documentos cargados a la lista
        all_docs.extend(docs)

# Imprimir la cantidad total de partes del PDF
print(f"Cantidad total de partes del PDF: {len(all_docs)}")

Cargando archivo: /content/pdf_files/respuestas.pdf
Cargando archivo: /content/pdf_files/teoria.pdf
Cargando archivo: /content/pdf_files/preguntas.pdf
Cantidad total de partes del PDF: 53


In [16]:
embeddings = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2') #modelo de embeddings gratis

In [17]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50) #usamos menos tokens que en el anterior debido a que este modelo es inferior
splits = text_splitter.split_documents(all_docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings, persist_directory='./vectordb')

retriever = vectorstore.as_retriever()

In [18]:
retriever.get_relevant_documents('¿Cuál es la idea básica del proyecto?')

[Document(metadata={'page': 0, 'source': '/content/pdf_files/respuestas.pdf'}, page_content='1.\nPresentación\ndel\nProyecto\n1.1.\nDescripción\ndel\nNegocio\nAyuda\npara\nResponder:\n●\nIdea\nBásica\ndel\nProyecto:\no\nPiensa\nen\nqué\noportunidad\nde\nmercado\nestás\naprovechando\ncon\ntu\nidea\nde \nnegocio.\nSegún\nel\ndocumento,\nla\nidea\ndebe\ncubrir\nuna\ndemanda\nno \nsatisfecha\no\nmejorar\nun\nproducto/servicio\nexistente.\no\nDescribe\nel\nproducto\no\nservicio\ndestacando\nsi\nes\nalgo\nnuevo\nen\nel \nmercado\no\nuna\ninnovación\nsobre\nalgo\nya\nexistente.\n●\nProductos\no\nServicios:\no\nSi\nes\nun\nproducto,\ndescribe\nsus\ncaracterísticas,'),
 Document(metadata={'page': 0, 'source': '/content/pdf_files/respuestas.pdf'}, page_content='1.\nPresentación\ndel\nProyecto\n1.1.\nDescripción\ndel\nNegocio\nAyuda\npara\nResponder:\n●\nIdea\nBásica\ndel\nProyecto:\no\nPiensa\nen\nqué\noportunidad\nde\nmercado\nestás\naprovechando\ncon\ntu\nidea\nde \nnegocio.\nSegún\nel\ndocume

In [19]:
import shutil

# Ruta de la carpeta que deseas comprimir
folder_path = '/content/vectordb'  # Cambia esto a la ruta de tu carpeta
zip_path = '/content/vectordb.zip'

# Comprimir la carpeta
shutil.make_archive('/content/vectordb', 'zip', folder_path)

'/content/vectordb.zip'